En este cuaderno vamos a generar los pares pregunta-respuesta, la idea es hacer un método genérico que acepte los nombres de los modelos y el se encargue de generar dichos pares y actualizar la base de datos con esta información.

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, LLaMATokenizer, LLaMAForCausalLM, GenerationConfig, pipeline, AutoConfig, pipeline
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, infer_auto_device_map
import torch
import torch, gc

# Método que carga el modelo tipo Bert de lenguaje y lo devuelve
def cargarModeloBert(m_lenguaje):
    
    base_model = m_lenguaje
    config = AutoConfig.from_pretrained(base_model)
    
    tokenizer = AutoTokenizer.from_pretrained(base_model)

    with init_empty_weights():
        model = AutoModelForCausalLM.from_config(config)

    model.tie_weights()
    device_map = infer_auto_device_map(model, no_split_module_classes=["GPTJBlock"], dtype="float16")
    model = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto", 
                                                 torch_dtype=torch.float16).cuda()
    
    return model, tokenizer

# Método que carga el modelo tipo MT5 de lenguaje y lo devuelve
def cargarModeloMT5(m_lenguaje):
    
    base_model = m_lenguaje
    config = AutoConfig.from_pretrained(base_model)
    
    tokenizer = AutoTokenizer.from_pretrained(base_model)

    with init_empty_weights():
        model = AutoModelForSeq2SeqLM.from_config(config)

    model.tie_weights()
    device_map = infer_auto_device_map(model, no_split_module_classes=["GPTJBlock"], dtype="float16")
    model = AutoModelForSeq2SeqLM.from_pretrained(base_model, device_map="auto", 
                                                 torch_dtype=torch.float16).cuda()
    
    return model, tokenizer

# Método que carga el modelo tipo LLaMA de lenguaje y lo devuelve
def cargarModeloLLaMA(m_lenguaje):
    
    base_model = m_lenguaje
    tokenizer =  LLaMATokenizer.from_pretrained(base_model)
    model = LLaMAForCausalLM.from_pretrained(base_model,
                                            device_map="auto",
                                            torch_dtype=torch.float16).cuda()

    return model, tokenizer

# En caso de que se desee hacer en inglés será necesario tener modelos de traducción
def cargarModelosTraducción():
    
    model_checkpoint_en_es = "Helsinki-NLP/opus-mt-en-es"
    model_checkpoint_es_en = "Helsinki-NLP/opus-mt-es-en"
    tra_en_es = pipeline("translation", model=model_checkpoint_en_es, max_length=1000)
    tra_es_en = pipeline("translation", model=model_checkpoint_es_en, max_length=1000)
    
    return tra_en_es, tra_es_en

def traducir(texto, traductor):
    return traductor(texto)[0]["translation_text"]

# Método para la generación de los pares de pregunta-respuesta
def generarParPR(contexto, modelo, tokenizer, idioma):
    if idioma=="esp":
        prompt = f"""Eres un asistente inteligente diseñado para generar preguntas y respuestas, dado el siguiente contexto:

        {contexto}

        A partir del texto anterior, genera un par pregunta-respuesta, es muy imporante que la pregunta comience por "PR:" 
        y la respuesta por "RP:" y, además, la pregunta siempre termine en salto de línea, es decir '\n'.
        """
    else:
        prompt = f"""You are an intelligent assistant designed to generate questions and answers, given the following context:

        {contexto}

        From the above text, generate a question-answer pair where the question starts with PR: and the answer starts with RP: and, in addition, the question always ends in a line break, i.e. '\n'.
        """
    
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = modelo.generate(
        input_ids=input_ids,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512, pad_token_id=tokenizer.eos_token_id
    )
    for seq in generation_output.sequences:
        output = tokenizer.decode(seq, skip_special_tokens=True)
    aux = output.split('\n')
    aux = [" ".join(x.split()) for x in aux]
    pregunta = aux[-2][3:]
    respuesta = aux[-1][3:]
    return pregunta, respuesta

# Metodo auxiliar para liberar la memoria de la GPU
def liberarGPU():
    gc.collect()
    with torch.no_grad():
        torch.cuda.empty_cache()
        
def comprobarGPULibre():
    t = torch.cuda.get_device_properties(0).total_memory
    a = torch.cuda.memory_allocated(0)
    return t-a

In [2]:
import pandas as pd

def engrosarCorpus(m_lenguaje, m_model, m_tokenizer, idioma="esp", ruta_contextos="../1-Base de datos documental/Contextos.csv", ruta_corpus="Corpus.csv"):
    
    # Comenzamos limpiando la memoria de la GPU, por si acaso
    liberarGPU()
    
    # Si el idioma es en inglés cargamos los modelos traductores
    if idioma=="eng":
        tra_en_es, tra_es_en = cargarModelosTraducción()
    
    # Cargamos los contextos sobre los que generaremos los pares p-r
    contextos = pd.read_csv(ruta_contextos, lineterminator='\n')
    
    # Cargamos el corpus donde añadiremos los nuevos pares generados
    try:
        corpus = pd.read_csv(ruta_corpus, lineterminator='\n')
    except FileNotFoundError:
        corpus = pd.DataFrame(columns=['Contexto', 'Documento', 'Pregunta', 'Respuesta', 'Modelo'])
        corpus.to_csv(ruta_corpus, sep=',', index=False)
    
    # Por último, generamos los pares preguntas respuestas:
    for index, row in contextos.iterrows():
        print(f"Fase {index} de {contextos.shape[0]}", end="\r")
        # De nuevo, si es en inglés necesitamos traducir
        if idioma=="eng":
            contexto_tra = traducir(row["Contexto"], tra_es_en)
            pregunta, respuesta = generarParPR(contexto_tra, m_model, m_tokenizer, idioma)
            pregunta, respuesta = traducir(pregunta, tra_en_es), traducir(pregunta, tra_en_es)
        else:
            pregunta, respuesta = generarParPR(row["Contexto"], m_model, m_tokenizer, idioma)
        corpus = pd.concat([corpus, pd.DataFrame([[row["Contexto"], row["Documento"], pregunta, respuesta, m_lenguaje]],
                                                 columns=['Contexto', 'Documento', 'Pregunta', 'Respuesta', 'Modelo'])], axis=0, ignore_index=True)
        corpus.to_csv("Corpus.csv", sep=',', index=False)
        
        # Guardamos un pequeño checkpoint de los contextos procesados
        if index%25 == 0:
            with open("Checkpoint.txt", 'w') as file:
                file.write(f"Fase {index} de {contextos.shape[0]}")

In [8]:
def engrosarCorpusTQG(m_lenguaje, m_model, ruta_contextos="../1-Base de datos documental/Contextos.csv", ruta_corpus="Corpus.csv"):
    # Comenzamos limpiando la memoria de la GPU, por si acaso
    liberarGPU()
    
    # Cargamos los contextos sobre los que generaremos los pares p-r
    contextos = pd.read_csv(ruta_contextos, lineterminator='\n')
    
    # Obtenemos los modelos de traducción ya que los modelos recibidos siempre actuarán en inglés
    tra_en_es, tra_es_en = cargarModelosTraducción()
    
    # Cargamos el corpus donde añadiremos los nuevos pares generados
    try:
        corpus = pd.read_csv(ruta_corpus, lineterminator='\n')
    except FileNotFoundError:
        corpus = pd.DataFrame(columns=['Contexto', 'Documento', 'Pregunta', 'Respuesta', 'Modelo'])
        corpus.to_csv(ruta_corpus, sep=',', index=False)
    
    # Por último, generamos los pares preguntas respuestas:
    for index, row in contextos.iterrows():
        print(f"Fase {index} de {contextos.shape[0]}", end="\r")
        
        try:
            # Generamos los pares con el modelo, es necesario traducir porque estos modelos son todos en inglés
            contexto_tra = traducir(row["Contexto"], tra_es_en)
            resultado = m_model.generate_qa(contexto_tra)
            for pregunta, respuesta in resultado:
                pregunta, respuesta = traducir(pregunta, tra_en_es), traducir(respuesta, tra_en_es)

            corpus = pd.concat([corpus, pd.DataFrame([[row["Contexto"], row["Documento"], pregunta, respuesta, m_lenguaje]],
                                                     columns=['Contexto', 'Documento', 'Pregunta', 'Respuesta', 'Modelo'])], axis=0, ignore_index=True)
            corpus.to_csv("Corpus.csv", sep=',', index=False)

            # Guardamos un pequeño checkpoint de los contextos procesados
            if index%25 == 0:
                with open("Checkpoint.txt", 'w') as file:
                    file.write(f"Fase {index} de {contextos.shape[0]}")
        except:
            print("Fila erronea")

In [4]:
m_lenguaje = "bertin-project/bertin-gpt-j-6B-alpaca"
engrosarCorpus(m_lenguaje, cargarModeloBert(m_lenguaje))

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
from lmqg import TransformersQG

m_lenguaje = 'lmqg/t5-base-squad-qag'
engrosarCorpusTQG(m_lenguaje, TransformersQG(m_lenguaje))

100%|██████████| 1/1 [00:00<00:00, 499.74it/s]


100%|██████████| 1/1 [00:00<00:00, 451.10it/s]


100%|██████████| 1/1 [00:00<00:00, 503.34it/s]


100%|██████████| 1/1 [00:00<00:00, 585.80it/s]


100%|██████████| 1/1 [00:00<00:00, 489.19it/s]


...

100%|██████████| 1/1 [00:00<00:00, 975.87it/s]


100%|██████████| 1/1 [00:00<00:00, 766.50it/s]


100%|██████████| 1/1 [00:00<00:00, 788.40it/s]


100%|██████████| 1/1 [00:00<00:00, 631.10it/s]


100%|██████████| 1/1 [00:00<00:00, 572.60it/s]
